In [1]:
import random
from os import truncate
from typing import Optional
import gymnasium as gym
import ale_py
import cv2
import numpy as np
from carcassone.carcassonne.carcassonne_game import CarcassonneGame
from carcassone.carcassonne.objects.actions.action import Action
from carcassone.carcassonne.tile_sets.supplementary_rules import SupplementaryRule
from carcassone.carcassonne.tile_sets.tile_sets import TileSet

# Środowiska
W ramach projektu zdecydowaliśmy się na wykorzystanie trzech środowisk o różnym stopniu trudności:
- udostępnionego przez OpenAI środowiska LunarLander;
- gry mobilnej 2048;
- gry planszowej Carcassone.

## Lunar Lander

Poprawka: Użyłabym tutaj jedną z https://ale.farama.org/environments/ bo bardziej pasują do gier - np. AlphaZero było testowane na nich

Lunar Lander to udostępnione przez OpenAI środowisko do optymalizacji trajektorii rakiety. Będzie ono dla nas środowiskiem testowym, ze względu na to, że jest dobrze znane i sprawdzone.

![](https://www.gymlibrary.dev/_images/lunar_lander.gif)

Źródła:
- https://www.gymlibrary.dev/environments/box2d/lunar_lander/

### Przykład użycia


In [7]:
env = gym.make(
    "LunarLander-v3",
    continuous = True,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
    render_mode="rgb_array"
)

state, _ = env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, _ = env.step(action)
    frame = env.render()
    cv2.imshow("frame", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    cv2.waitKey(50)

cv2.destroyAllWindows()

In [4]:
gym.register_envs(ale_py)

env = gym.make('ALE/MsPacman-v5', render_mode='rgb_array')

state, _ = env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action = env.action_space.sample()
    next_state, reward, terminated, truncated, _ = env.step(action)
    frame = env.render()
    cv2.imshow("frame", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    cv2.waitKey(50)

cv2.destroyAllWindows()

## 2048

2048 to popularna gra mobilna, polegająca na przesuwaniu i łączeniu liczb w celu uzyskania klocka 2048.

![](https://upload.wikimedia.org/wikipedia/commons/6/64/2048_Screenshot.png)

Źródła:
- https://flothesof.github.io/2048-game.html
- https://github.com/qwert12500/2048_rl

### Przykładowa rozgrywka

In [3]:
from game2048.board2048 import Board2048

def main_loop(b, direction):
    new_board = b.move(direction)
    moved = False
    if (new_board == b.board).all():
        # move is invalid
        pass
    else:
        moved = True
        b.board = new_board
        b.fill_cell()
    return moved

game = Board2048()
finish = False
while not finish:
  direction = np.random.randint(4)
  if direction < 0 or direction > 3:
    continue
  moved = main_loop(game, direction)
  if not moved:
    continue
  print(game.board)
  print(game.total_score)
  finish = game.is_game_over()
print("Game Over!, Total Score is {}".format(game.total_score))

[[0 0 0 4]
 [0 0 0 2]
 [0 0 0 0]
 [0 0 0 0]]
0
[[4 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 2 0]]
0
[[0 0 0 0]
 [0 0 0 0]
 [4 0 0 2]
 [2 0 2 0]]
0
[[4 0 2 2]
 [2 0 0 0]
 [2 0 0 0]
 [0 0 0 0]]
0
[[4 4 0 0]
 [2 4 0 0]
 [2 0 0 0]
 [0 0 0 0]]
4
[[8 0 0 0]
 [2 4 0 0]
 [2 0 0 0]
 [0 0 2 0]]
12
[[0 0 0 8]
 [0 0 2 4]
 [0 0 2 2]
 [0 0 0 2]]
12
[[2 0 4 8]
 [0 0 0 4]
 [0 0 0 4]
 [0 0 0 0]]
20
[[2 4 8 0]
 [4 2 0 0]
 [4 0 0 0]
 [0 0 0 0]]
20
[[0 2 4 8]
 [0 0 4 2]
 [0 2 0 4]
 [0 0 0 0]]
20
[[0 4 8 8]
 [2 0 0 2]
 [0 0 0 4]
 [0 0 0 0]]
32
[[ 0  0  4 16]
 [ 2  0  0  4]
 [ 0  0  0  4]
 [ 0  0  0  0]]
52
[[ 4 16  0  0]
 [ 2  4  0  2]
 [ 4  0  0  0]
 [ 0  0  0  0]]
52
[[ 0  0  0  0]
 [ 4  2  0  0]
 [ 2 16  0  0]
 [ 4  4  0  2]]
52
[[ 0  0  0  0]
 [ 4  2  0  2]
 [ 2 16  0  0]
 [ 8  2  0  0]]
60
[[ 4  2  0  2]
 [ 2 16  0  0]
 [ 8  2  0  0]
 [ 2  0  0  0]]
60
[[ 4  4  0  0]
 [ 2 16  0  0]
 [ 8  2  0  0]
 [ 2  0  2  0]]
64
[[ 8  0  0  0]
 [ 2 16  0  0]
 [ 8  2  0  0]
 [ 4  0  2  0]]
76
[[ 0  0  0  8]
 [ 0  0  2 16]

## Carsassone

Carcassone to popularna gra planszowa, polegająca na układaniu bloków z miastami, drogami oraz klasztorami.

![](https://upload.wikimedia.org/wikipedia/commons/d/d7/Deskohran%C3%AD_08-10-05_006.jpg)

Źródła:
- https://github.com/wingedsheep/carcassonne
- https://wingedsheep.com/programming-carcassonne/

### Przykładowa rozgrywka

In [1]:
game = CarcassonneGame(
	players=2,
	tile_sets=[TileSet.BASE, TileSet.THE_RIVER, TileSet.INNS_AND_CATHEDRALS],
	supplementary_rules=[SupplementaryRule.ABBOTS, SupplementaryRule.FARMERS]
)

while not game.is_finished():
    player: int = game.get_current_player()
    valid_actions: [Action] = game.get_possible_actions()
    action: Optional[Action] = random.choice(valid_actions)
    if action is not None:
        game.step(player, action)
    game.render()

NameError: name 'CarcassonneGame' is not defined